In [1]:
import h5py

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# models
discrete_encoder_path = 'discrete_encoder.pth'
discrete_decoder_path = 'discrete_decoder.pth'
mapping_path = 'mapping.pth'
unet_path = 'con_unet_full.pth'
continuous_model_path = 'continuous_model.pth'

hidden_dim_discrete = 128

# data preparation
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/sdf_train_core.h5'
sample_size = 2048
batch_size = 32
split_ratio = 0.9
depth_size = 256
num_of_workers = 12
# training
num_epoch = 40

In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
import torch.nn as nn
import torchvision.transforms.functional as TF
from torch.autograd import Variable
from PIL import Image
import sys
import time
import matplotlib.pyplot as plt
from torchsummary import summary

In [5]:
# reproducible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)
np.random.seed(0)

In [6]:
device

device(type='cuda')

In [7]:
class ChairSDFDataset(Dataset):
     
    def __init__(self, h5_file):
        
        self.file_path = h5_file
        self.dataset = None
        
        with h5py.File(self.file_path, 'r') as file:
            self.dataset_len = len(file)
            self.keys = list(file.keys())
            
        self.to_tensor = transforms.ToTensor()
     
    def __len__(self):
        return self.dataset_len
 
    def __getitem__(self, idx):
        
        #start_time = time.time()
        
        if self.dataset is None:
            self.dataset = h5py.File(self.file_path, 'r')
         
        group = self.dataset[self.keys[idx]]
        
        depth_img = self.to_tensor(Image.fromarray(np.array(group['depth_img'])))
        
        #print("--- depth preprocessing %s seconds ---" % (time.time() - start_time))
        
        sample_pt_np = np.array(group['sample_pt']).reshape(-1, 3)
        sample_sdf_np = np.array(group['sample_sdf']).reshape(-1, 1)
        
        # check size correctness and fix incorrect data
        if sample_pt_np.shape[0] != 2048:
            sample_pt_np = np.pad(sample_pt_np, ((0, 2048 - sample_pt_np.shape[0]), (0, 0)), 'reflect')
        if sample_sdf_np.shape[0] != 2048:
            sample_sdf_np = np.pad(sample_sdf_np, ((0, 2048 - sample_sdf_np.shape[0]), (0, 0)), 'reflect')
            
        
        sample_pt = torch.from_numpy(sample_pt_np).float()
        sample_sdf = torch.from_numpy(sample_sdf_np).float()
        # scale sdf
        sample_sdf = torch.sign(sample_sdf)*torch.pow(torch.abs(sample_sdf), 0.25)
        
        #print("--- subsampling %s seconds ---" % (time.time() - start_time))
        
        target_vox = torch.from_numpy(np.array(group['target_vox'])).float()
        
        sample = { 'depth_img': depth_img,
                   'sample_pt':sample_pt,
                   'sample_sdf':sample_sdf,
                   'target_vox':target_vox,
                  }
        
        return sample

In [8]:
train_sdf_dataset = ChairSDFDataset(data_file)

train_sdf_dataloader = DataLoader(train_sdf_dataset, batch_size=batch_size, shuffle=True, num_workers=num_of_workers)

In [9]:
len(train_sdf_dataloader)

2634

In [10]:
start_epoch = 0
latent_dim = 256

from models import Discrete_encoder, Mapping, Discrete_decoder, Conditional_UNET

####################
# Discrete Encoder #
####################

discrete_encoder = Discrete_encoder(256).to(device)
discrete_encoder.load_state_dict(torch.load(discrete_encoder_path))
discrete_encoder.eval()

for child in discrete_encoder.children():
    for param in child.parameters():
        param.requires_grad = False


####################
# Mapping #
####################

mapping = Mapping().to(device)
mapping.load_state_dict(torch.load(mapping_path))
mapping.eval()

for child in mapping.children():
    for param in child.parameters():
        param.requires_grad = False
        
# ####################
# # Discrete Decoder #
# ####################
        
discrete_decoder = Discrete_decoder(256).to(device)
discrete_decoder.load_state_dict(torch.load(discrete_decoder_path))
discrete_decoder.eval()

for child in discrete_decoder.children():
    for param in child.parameters():
        param.requires_grad = False
        
########
# UNET #
########

# pre-trained model is loaded within the model
unet = Conditional_UNET(unet_path).to(device)
unet.eval()

for child in unet.children():
    for param in child.parameters():
        param.requires_grad = False

In [11]:
class BottleNect1D(nn.Module):
    def __init__(self, input_dim, expand = 5):
        super(BottleNect1D, self).__init__()
        
        self.block = nn.Sequential(
            nn.Linear(input_dim, expand*input_dim),
            nn.BatchNorm1d(expand*input_dim),
            nn.ReLU(),
            nn.Linear(expand*input_dim, input_dim),
            nn.BatchNorm1d(input_dim),
        )
    
    def forward(self, x):
        return x + self.block(x)

class Continuous(nn.Module):
    def __init__(self, pt_dim = 3, con_dim = 32, latent_dim = 256):
        super(Continuous, self).__init__()
        
        self.de_pt =  nn.Sequential(
            nn.Linear(pt_dim + con_dim + latent_dim, latent_dim),
            nn.BatchNorm1d(latent_dim),
            BottleNect1D(latent_dim),
        )
        
        self.de_1 = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.BatchNorm1d(latent_dim),
            BottleNect1D(latent_dim),
        )
        
        self.de_2 = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.BatchNorm1d(latent_dim),
            BottleNect1D(latent_dim),
            nn.Linear(latent_dim, 1),
            nn.Tanh(),
        )
        
    def forward(self, pt, con, z):
        
        fea = self.de_pt(torch.cat((torch.cat((pt, con), 1), z), 1))
        out = self.de_1(fea) + fea
        out = self.de_2(out)
        
        return out

In [12]:
continuous = Continuous().to(device)
continuous.load_state_dict(torch.load(continuous_model_path))

<All keys matched successfully>

In [13]:
summary(continuous, [(3, ), (32, ), (256, )])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]          74,752
       BatchNorm1d-2                  [-1, 256]             512
            Linear-3                 [-1, 1280]         328,960
       BatchNorm1d-4                 [-1, 1280]           2,560
              ReLU-5                 [-1, 1280]               0
            Linear-6                  [-1, 256]         327,936
       BatchNorm1d-7                  [-1, 256]             512
      BottleNect1D-8                  [-1, 256]               0
            Linear-9                  [-1, 256]          65,792
      BatchNorm1d-10                  [-1, 256]             512
           Linear-11                 [-1, 1280]         328,960
      BatchNorm1d-12                 [-1, 1280]           2,560
             ReLU-13                 [-1, 1280]               0
           Linear-14                  [

In [14]:
l1loss = nn.L1Loss()

In [15]:
lr = 1e-2

model = continuous

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [16]:
def save_checkpoint(state):
    torch.save(state, 'continuous_model_checkpoint.pth.tar')

In [17]:
checkpoint = torch.load('continuous_model_checkpoint.pth.tar')
start_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['continuous_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [18]:
# advanced indexing 2x2x2 context from voxel
def getContext(sample_pt_query, vox):
    
    # sample_pt bxmx3
    # vox bxcxdimxdimxdim
    
    channel_size = vox.shape[1]
    batch_size, sample_size, _ = sample_pt_query.shape
    meshgrid_base = torch.Tensor(np.meshgrid(np.arange(0, batch_size), np.arange(0, channel_size), np.arange(0, 2), np.arange(0, 2), np.arange(0, 2))).int()
    context = torch.empty((batch_size, sample_size, channel_size, 2, 2, 2))
    
    for j in range(context.shape[1]):
        context[:, j, :, :, :, :] = vox[
                    meshgrid_base[0].long(),
                    meshgrid_base[1].long(),
                    (meshgrid_base[2] + sample_pt_query[:, j, 0].reshape(1, -1, 1, 1, 1)).long(), 
                    (meshgrid_base[3] + sample_pt_query[:, j, 1].reshape(1, -1, 1, 1, 1)).long(), 
                    (meshgrid_base[4] + sample_pt_query[:, j, 2].reshape(1, -1, 1, 1, 1)).long()
                ].transpose(0, 1)
    
    # b x c x m x 2 x 2 x 2
    return context.transpose(1, 2)

In [19]:
def trilinearInterpolation(context, dx, dy, dz):
    
    v0 = context[:, :, :, 0, 0, 0]*(1-dx)*(1-dy)*(1-dz)
    v1 = context[:, :, :, 1, 0, 0]*dx*(1-dy)*(1-dz)
    v2 = context[:, :, :, 0, 1, 0]*(1-dx)*dy*(1-dz)
    v3 = context[:, :, :, 1, 1, 0]*dx*dy*(1-dz)
    v4 = context[:, :, :, 0, 0, 1]*(1-dx)*(1-dy)*dz
    v5 = context[:, :, :, 1, 0, 1]*dx*(1-dy)*dz
    v6 = context[:, :, :, 0, 1, 1]*(1-dx)*dy*dz
    v7 = context[:, :, :, 1, 1, 1]*dx*dy*dz
    
    # b x c x m 1
    return v0 + v1 + v2 + v3 + v4 + v5 + v6 + v7

In [20]:
#########################################
#test sub module

import time

vox_size = 32
latent_dim = 256
con_dim = 32

batch_len = len(train_sdf_dataloader)

print("Starting Training Loop...")

start_time = time.time()

for epoch in range(start_epoch, num_epoch):
    
    loss_list = []
    loss_batch = []
    
    print("Epoch: ", epoch)
    
    count = 0
    
    for i, data in enumerate(train_sdf_dataloader):
        
        ####################
        # Data preparation #
        ####################
        
        # b x 1 x 256 x 256
        depth_img = data['depth_img'].to(device)
        # b x 128 x 1 x 1
        z = discrete_encoder(depth_img)
        # b x n x 3
        # DO NOT scale by np.sqrt(3)
        sample_pt = data['sample_pt']
        # b x n x 1
        sample_sdf = data['sample_sdf']
        
        # b x 16 x 64 x 64 x 64
        target_vox = data['target_vox'].to(device)
        vox_feature = unet(target_vox, z)
        #vox_feature = unet(torch.sigmoid(discrete_decoder(mapping(z))))
        
        ####################
        # indexing context #
        ####################
        
        # stay with cpu for v-ram efficiency
        sample_pt_normalized = sample_pt + torch.tensor([0.5, 0.5, 0.5])
        # (0, vox_size-1)
        sample_pt_scale = torch.clamp(sample_pt_normalized* (vox_size-1), 0, (vox_size-1)-1e-5)
        # (0, vox_size-2)
        sample_pt_query = torch.clamp((sample_pt_scale).int(), 0, (vox_size-2))
        sample_pt_distance = sample_pt_scale - sample_pt_query
        
        context = getContext(sample_pt_query, vox_feature)
        
        dx = sample_pt_distance[:, :, 0].unsqueeze(1)
        dy = sample_pt_distance[:, :, 1].unsqueeze(1)
        dz = sample_pt_distance[:, :, 2].unsqueeze(1)
        # local feature
        con = trilinearInterpolation(context, dx, dy, dz)
        
        ################################
        # Reshape input & forward pass #
        ################################
        
        sample_pt = sample_pt.transpose(-1, -2).to(device)
        con = con.to(device)
        z = z.squeeze(-1).squeeze(-1).repeat(1, 1, sample_size)
        sample_sdf = sample_sdf.transpose(-1, -2).to(device)
        
        
        sample_pt = sample_pt.transpose(-1, -2).reshape(-1, 3)
        con = con.transpose(-1, -2).reshape(-1, con_dim)
        z = z.transpose(-1, -2).reshape(-1, latent_dim)
        sample_sdf = sample_sdf.transpose(-1, -2).reshape(-1, 1)
        
        
        optimizer.zero_grad()
        
        pred_sdf = model(sample_pt, con, z)
        
        loss_l1 = l1loss(pred_sdf, sample_sdf)
        
        loss = loss_l1
        
        loss.backward()
        
        loss_list.append(loss_l1.item())
        loss_batch.append(loss_l1.item())
        
        optimizer.step()
        
        #scheduler.step()
        
        if count != 0 and count % 10 == 0:
            loss_batch_avg = np.average(loss_batch)
            
            print("Batch: ", count, ", l1 Loss: ", loss_batch_avg, ", Time: %s s" % (time.time() - start_time))
            
            if count % 500 == 0:
                torch.save(model.state_dict(), continuous_model_path)
                
            loss_batch.clear()
            
        count += 1
        
    print("Epoch: ", epoch, ', l1 loss: ', np.average(loss_list))
    
    loss_list.clear()
    
    torch.save(model.state_dict(), continuous_model_path)
    
    save_checkpoint({
        'epoch': epoch + 1,
        'continuous_state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    })
    
print("Training finished")

Starting Training Loop...
Epoch:  5
Batch:  10 , l1 Loss:  0.09730378877032887 , Time: 12.547937154769897 s
Batch:  20 , l1 Loss:  0.09698976129293442 , Time: 23.788269519805908 s
Batch:  30 , l1 Loss:  0.09679423198103905 , Time: 35.15193963050842 s
Batch:  40 , l1 Loss:  0.09688738137483596 , Time: 46.41036248207092 s
Batch:  50 , l1 Loss:  0.0968507707118988 , Time: 57.68720531463623 s
Batch:  60 , l1 Loss:  0.0967412270605564 , Time: 69.05775380134583 s
Batch:  70 , l1 Loss:  0.09695733115077018 , Time: 80.44469022750854 s
Batch:  80 , l1 Loss:  0.09679867178201676 , Time: 91.75320625305176 s
Batch:  90 , l1 Loss:  0.096273422986269 , Time: 103.14655542373657 s
Batch:  100 , l1 Loss:  0.09636975228786468 , Time: 114.53690457344055 s
Batch:  110 , l1 Loss:  0.09654539600014686 , Time: 126.00327825546265 s
Batch:  120 , l1 Loss:  0.09665432870388031 , Time: 137.41813969612122 s
Batch:  130 , l1 Loss:  0.0969502091407776 , Time: 148.86789774894714 s
Batch:  140 , l1 Loss:  0.097324442

Batch:  1140 , l1 Loss:  0.09692788273096084 , Time: 1341.4211497306824 s
Batch:  1150 , l1 Loss:  0.09703014120459556 , Time: 1353.3935952186584 s
Batch:  1160 , l1 Loss:  0.09717827960848809 , Time: 1365.3745381832123 s
Batch:  1170 , l1 Loss:  0.09708421900868416 , Time: 1377.3647546768188 s
Batch:  1180 , l1 Loss:  0.09639296978712082 , Time: 1389.3431260585785 s
Batch:  1190 , l1 Loss:  0.09621585384011269 , Time: 1401.3016185760498 s
Batch:  1200 , l1 Loss:  0.09657689854502678 , Time: 1413.2541165351868 s
Batch:  1210 , l1 Loss:  0.09617431312799454 , Time: 1425.2734820842743 s
Batch:  1220 , l1 Loss:  0.09665863364934921 , Time: 1437.2488913536072 s
Batch:  1230 , l1 Loss:  0.09819706454873085 , Time: 1449.2113966941833 s
Batch:  1240 , l1 Loss:  0.09746557921171188 , Time: 1461.1953434944153 s
Batch:  1250 , l1 Loss:  0.09744667336344719 , Time: 1473.172420501709 s
Batch:  1260 , l1 Loss:  0.09676851928234101 , Time: 1485.1560673713684 s
Batch:  1270 , l1 Loss:  0.096139840036

Batch:  2260 , l1 Loss:  0.09657721146941185 , Time: 2672.1697809696198 s
Batch:  2270 , l1 Loss:  0.09640543311834335 , Time: 2683.9986712932587 s
Batch:  2280 , l1 Loss:  0.09686446934938431 , Time: 2695.8095605373383 s
Batch:  2290 , l1 Loss:  0.09697410762310028 , Time: 2707.6153678894043 s
Batch:  2300 , l1 Loss:  0.09656915962696075 , Time: 2719.4463455677032 s
Batch:  2310 , l1 Loss:  0.09688675105571747 , Time: 2731.2908782958984 s
Batch:  2320 , l1 Loss:  0.09631935656070709 , Time: 2743.2073888778687 s
Batch:  2330 , l1 Loss:  0.09780078232288361 , Time: 2755.080888032913 s
Batch:  2340 , l1 Loss:  0.09752569571137429 , Time: 2766.9420149326324 s
Batch:  2350 , l1 Loss:  0.09737838059663773 , Time: 2778.795274734497 s
Batch:  2360 , l1 Loss:  0.09638497158885002 , Time: 2790.652596473694 s
Batch:  2370 , l1 Loss:  0.09726045578718186 , Time: 2802.502907514572 s
Batch:  2380 , l1 Loss:  0.09620549455285073 , Time: 2814.34779047966 s
Batch:  2390 , l1 Loss:  0.09585637226700783

Batch:  760 , l1 Loss:  0.09638672024011612 , Time: 4027.142274618149 s
Batch:  770 , l1 Loss:  0.09569787681102752 , Time: 4039.339052438736 s
Batch:  780 , l1 Loss:  0.09581379219889641 , Time: 4051.5288376808167 s
Batch:  790 , l1 Loss:  0.09635413363575936 , Time: 4063.7491171360016 s
Batch:  800 , l1 Loss:  0.09714079201221466 , Time: 4075.9154908657074 s
Batch:  810 , l1 Loss:  0.09674498438835144 , Time: 4088.0523879528046 s
Batch:  820 , l1 Loss:  0.09643762707710266 , Time: 4100.180474996567 s
Batch:  830 , l1 Loss:  0.0961882896721363 , Time: 4112.240678071976 s
Batch:  840 , l1 Loss:  0.09607253819704056 , Time: 4124.271617174149 s
Batch:  850 , l1 Loss:  0.0963285133242607 , Time: 4136.381882190704 s
Batch:  860 , l1 Loss:  0.09631945937871933 , Time: 4148.54926776886 s
Batch:  870 , l1 Loss:  0.0956260770559311 , Time: 4160.725952386856 s
Batch:  880 , l1 Loss:  0.09604616314172745 , Time: 4172.865138053894 s
Batch:  890 , l1 Loss:  0.09627354890108109 , Time: 4185.1249897

Batch:  1890 , l1 Loss:  0.09667198956012726 , Time: 5381.261624574661 s
Batch:  1900 , l1 Loss:  0.09562738984823227 , Time: 5393.257243871689 s
Batch:  1910 , l1 Loss:  0.09635978788137436 , Time: 5405.172685146332 s
Batch:  1920 , l1 Loss:  0.09586903303861619 , Time: 5417.092921495438 s
Batch:  1930 , l1 Loss:  0.09617278650403023 , Time: 5429.050471305847 s
Batch:  1940 , l1 Loss:  0.09566814079880714 , Time: 5440.989052772522 s
Batch:  1950 , l1 Loss:  0.0962630107998848 , Time: 5452.929258584976 s
Batch:  1960 , l1 Loss:  0.0960507795214653 , Time: 5464.821830511093 s
Batch:  1970 , l1 Loss:  0.09582269340753555 , Time: 5476.826341152191 s
Batch:  1980 , l1 Loss:  0.09658411219716072 , Time: 5488.763051748276 s
Batch:  1990 , l1 Loss:  0.09609104692935944 , Time: 5500.850157499313 s
Batch:  2000 , l1 Loss:  0.09594937488436699 , Time: 5512.812280893326 s
Batch:  2010 , l1 Loss:  0.0954067163169384 , Time: 5524.680714130402 s
Batch:  2020 , l1 Loss:  0.09647263288497925 , Time: 5

...

Batch:  2300 , l1 Loss:  0.09216807335615158 , Time: 59799.606085538864 s
Batch:  2310 , l1 Loss:  0.09268941804766655 , Time: 59811.55937218666 s
Batch:  2320 , l1 Loss:  0.09283859431743621 , Time: 59823.53705692291 s
Batch:  2330 , l1 Loss:  0.09309417754411697 , Time: 59835.47858667374 s
Batch:  2340 , l1 Loss:  0.09208760857582092 , Time: 59847.45761299133 s
Batch:  2350 , l1 Loss:  0.09253058433532715 , Time: 59859.535358667374 s
Batch:  2360 , l1 Loss:  0.09284318163990975 , Time: 59871.62615060806 s
Batch:  2370 , l1 Loss:  0.09331469833850861 , Time: 59883.57469725609 s
Batch:  2380 , l1 Loss:  0.09300043582916259 , Time: 59895.61374640465 s
Batch:  2390 , l1 Loss:  0.09302884638309479 , Time: 59907.60948777199 s
Batch:  2400 , l1 Loss:  0.0931331530213356 , Time: 59919.60476517677 s
Batch:  2410 , l1 Loss:  0.09300384670495987 , Time: 59931.555278778076 s
Batch:  2420 , l1 Loss:  0.09234908074140549 , Time: 59943.557394742966 s
Batch:  2430 , l1 Loss:  0.09236533492803574 , T

Batch:  800 , l1 Loss:  0.09295364320278168 , Time: 61169.58041334152 s
Batch:  810 , l1 Loss:  0.09296929389238358 , Time: 61181.885290145874 s
Batch:  820 , l1 Loss:  0.09232843741774559 , Time: 61194.15047240257 s
Batch:  830 , l1 Loss:  0.09224009662866592 , Time: 61206.36690855026 s
Batch:  840 , l1 Loss:  0.09267477616667748 , Time: 61218.581325531006 s
Batch:  850 , l1 Loss:  0.09181154519319534 , Time: 61230.774414777756 s
Batch:  860 , l1 Loss:  0.09293441846966743 , Time: 61243.00513839722 s
Batch:  870 , l1 Loss:  0.09256464913487435 , Time: 61255.18798995018 s
Batch:  880 , l1 Loss:  0.09277681410312652 , Time: 61267.41734910011 s
Batch:  890 , l1 Loss:  0.09234129264950752 , Time: 61279.61196398735 s
Batch:  900 , l1 Loss:  0.09257749989628791 , Time: 61291.82524442673 s
Batch:  910 , l1 Loss:  0.09225644916296005 , Time: 61304.105217695236 s
Batch:  920 , l1 Loss:  0.09293337538838387 , Time: 61316.29985547066 s
Batch:  930 , l1 Loss:  0.09273690804839134 , Time: 61328.53

Batch:  1930 , l1 Loss:  0.09269302934408188 , Time: 62521.5199201107 s
Batch:  1940 , l1 Loss:  0.09250234663486481 , Time: 62533.44794487953 s
Batch:  1950 , l1 Loss:  0.09265127778053284 , Time: 62545.38229250908 s
Batch:  1960 , l1 Loss:  0.0931091584265232 , Time: 62557.314366817474 s
Batch:  1970 , l1 Loss:  0.09288654550909996 , Time: 62569.22805809975 s
Batch:  1980 , l1 Loss:  0.0924242004752159 , Time: 62581.177762269974 s
Batch:  1990 , l1 Loss:  0.09262823015451431 , Time: 62593.3257932663 s
Batch:  2000 , l1 Loss:  0.09190483838319778 , Time: 62605.51591396332 s
Batch:  2010 , l1 Loss:  0.09211397916078568 , Time: 62617.79542636871 s
Batch:  2020 , l1 Loss:  0.09292435944080353 , Time: 62630.05665397644 s
Batch:  2030 , l1 Loss:  0.09288075491786003 , Time: 62642.248975515366 s
Batch:  2040 , l1 Loss:  0.09228165224194526 , Time: 62654.356796979904 s
Batch:  2050 , l1 Loss:  0.09208830446004868 , Time: 62666.46576881409 s
Batch:  2060 , l1 Loss:  0.0926701232790947 , Time:

Batch:  430 , l1 Loss:  0.09265918880701066 , Time: 63892.338023900986 s
Batch:  440 , l1 Loss:  0.09241940602660179 , Time: 63904.25314068794 s
Batch:  450 , l1 Loss:  0.09278726503252983 , Time: 63916.191086769104 s
Batch:  460 , l1 Loss:  0.09294340834021568 , Time: 63928.0170044899 s
Batch:  470 , l1 Loss:  0.09209810122847557 , Time: 63940.4781126976 s
Batch:  480 , l1 Loss:  0.09252860248088837 , Time: 63952.650946855545 s
Batch:  490 , l1 Loss:  0.09238251745700836 , Time: 63964.56271314621 s
Batch:  500 , l1 Loss:  0.09218742698431015 , Time: 63976.572607040405 s
Batch:  510 , l1 Loss:  0.09175013527274131 , Time: 63988.802139759064 s
Batch:  520 , l1 Loss:  0.09230774641036987 , Time: 64000.987428188324 s
Batch:  530 , l1 Loss:  0.0923464022576809 , Time: 64013.24747920036 s
Batch:  540 , l1 Loss:  0.09265836477279663 , Time: 64025.46332383156 s
Batch:  550 , l1 Loss:  0.09174738749861718 , Time: 64037.67301988602 s
Batch:  560 , l1 Loss:  0.0922589048743248 , Time: 64049.9028

KeyboardInterrupt: 